In [1]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
actions = {
    'Hello',
    'Meet',
    'NiceTMY',
    'I',
    'Name'
}

data = np.concatenate([
    np.load('dataset/seq_Hello_1715840691.npy'),
    np.load('dataset/seq_Meet_1715840691.npy'),
    np.load('dataset/seq_NiceTMY_1715840691.npy'),
    np.load('dataset/seq_I_1715840691.npy'),
    np.load('dataset/seq_Name_1715840691.npy')
], axis=0)

data.shape

(3454, 30, 100)

In [3]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(3454, 30, 99)
(3454,)


In [4]:
import tensorflow as tf

num_classes = len(actions)

y_data = tf.one_hot(labels.astype(np.int32), depth=num_classes)
print(y_data.shape)

(3454, 5)


In [5]:
from sklearn.model_selection import train_test_split

# Convert data to NumPy arrays
x_data = x_data.astype(np.float32)
y_data = y_data.numpy()  # Convert tensor to NumPy array

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(3108, 30, 99) (3108, 5)
(346, 30, 99) (346, 5)


In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

model = Sequential([
    Input(shape=x_train.shape[1:3]),
    LSTM(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        41,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,229 (172.77 KB)

 Trainable params: 44,229 (172.77 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import load_model

# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)

# Save the model
model.save('models/model.h5')

model_path = 'models/model.h5'
model = load_model(model_path, compile=False)

y_pred = model.predict(x_val)

# Convert predictions and true labels to integer indices
y_val_int = np.argmax(y_val, axis=1).astype(np.int32)
y_pred_int = np.argmax(y_pred, axis=1).astype(np.int32)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_val_int, y_pred_int)
print(conf_matrix)

# Classification report
class_report = classification_report(y_val_int, y_pred_int, target_names=actions)
print(class_report)
# def multilabel_confusion_matrix(y_true, y_pred):
#     pass
# 
# 
# multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))

ImportError: cannot import name 'load_model' from 'tensorflow' (C:\Users\hansb\PycharmProjects\HandTracking\venv\Lib\site-packages\tensorflow\__init__.py)